In [22]:
import pandas as pd
import numpy as np
from uuid import uuid4
pd.options.display.max_columns = None

In [44]:
master = pd.read_csv('../../data/Cleaned-11-Jan-22.csv')
master.drop_duplicates(inplace=True)

/var/folders/gb/1cwn6rds3vv7p30s87jjmp200000gn/T/ipykernel_65131/226527222.py:1: DtypeWarning: Columns (24,26,28,29,30,31,32,33,34,35,36,37,38,40,41,43,46,47,48,49,50,51,52,53,63) have mixed types. Specify dtype option on import or set low_memory=False.
  master = pd.read_csv('../../data/Cleaned-11-Jan-22.csv')


In [3]:
# Style ids that have more than one set of S,M,L

unique_style_id = {}
issues = set()
for index, row in master.iterrows():
    unique_key = row['style_id'] + '>' + row['primary_colour']
    if unique_key not in unique_style_id.keys():
        unique_style_id[unique_key] = [row['standard_size']]
    else:
        if row['standard_size'] in unique_style_id[unique_key]:
            issues.add(unique_key)
        unique_style_id[unique_key].append(row['standard_size'])
issues

set()

In [45]:
for style in unique_style_id:
    unique_style_id[style] = str(uuid4())
unique_style_id

{'TIG-0720-WCT24-01>Grey': 'c7452bfc-b016-4887-9072-481a5b0f3dd2',
 'TIG-0720-WCT24-02>Orange & White': 'ce81f27b-5d1b-47a7-8887-0dd8e81fdd2e',
 'TIG-0720-WCT22-05>Multicolour': '21a41660-30d1-427d-a373-18e6e81b2caf',
 '1119-CHINO04-07>Lt. Grey': '0a118ad4-174c-47e3-a0e1-1cbfe1b2b612',
 '1119-CHINO04-06>Teal': '96ac5c9e-55a1-4d88-bfd3-063b593242d8',
 '0920SHORTS02-13>Lt. Grey': 'a1e4417d-7e4e-4af1-80a6-df78dc4b51a6',
 '0920SHORTS02-14>Teal': '194f7fc3-38ef-46f5-9ac1-d0b393da14ad',
 '1120-SH57-06>Beige/Navy': 'eea5c9ac-1498-46a9-92e5-828e7e8e9e3e',
 '1120-SH57-07>Grey': '4cd85792-a46a-435a-bdc8-baf628ac8c0b',
 '1120-SH57-08>Grey': '7fa294bc-db19-436b-ade7-93a94a1459b0',
 '1120-SH57-09>Navy': 'c0e7c911-766f-4f81-8d9e-3e9b14eb64d9',
 '1120-SH57-10>Grey': '7111a82c-4aef-43ac-9abe-37aacb46cb75',
 '1120-SH57-11>Navy': 'b4058245-08df-40d6-8941-3df4cad0f8d6',
 '1120-SH57-12>Black': 'c91d3399-dfad-4209-a738-f0e745a1fee7',
 '1120-SH57-13>Olive Green': 'b4321fca-1150-4361-8688-7566bbb19bc7',
 '11

In [46]:
for index, row in master.iterrows():
    master.at[index, 'product_id'] = unique_style_id[row['style_id'] + '>' + row['primary_colour']]

In [47]:
# read the file that has colours given by tigc
tigc_colours = pd.read_csv('Colours - tigc colours.csv')
all_colours = set()
all_colours.update(tigc_colours['Barcode Colour'].unique())
all_colours.update(master['primary_colour'].unique())

unique_colours = set()
for colour in all_colours:
    colour = colour.replace('&', '/').split('/')
    if type(colour) == list:
        for c in colour:
            unique_colours.add(c.strip().lower())
    else:
        unique_colours.add(colour.strip().lower())

# export unique colors (they have spelling errors)
# pd.DataFrame(list(unique_colours)).to_csv('unique_colors.csv', index=False)

In [48]:
# read the unique colours file with spelling errors fixed
change_color = pd.read_csv('new colors clean - unique_colors.csv')
change_color = change_color[change_color['new'].isnull()==False]
change_color_dict = {}
for index, row in change_color.iterrows():
    change_color_dict[row["old"]] = row["new"]

clean_unique_colours = set()
for colour in unique_colours:
    if change_color_dict.get(colour):
        clean_unique_colours.add(change_color_dict[colour])
    else:
        clean_unique_colours.add(colour)

# export unique & correct spelling colours
# pd.DataFrame(list(clean_unique_colours)).to_csv('clean_unique_colors.csv', index=False)

In [49]:
# read the file with unique clean colours and their families
col_fam = pd.read_csv('new colors clean - Sheet1.csv')

# create primary, secondary, colour family columns
for index, row in master.iterrows():
    clean_colours = []
    colours = row['primary_colour'].replace('&', '/').split('/')
    if type(colours) == str:
        colours = [colours]
    for col in colours:
        col = col.strip().lower()
        if change_color_dict.get(col):
            col = change_color_dict[col]
        clean_colours.append(col)
    if len(clean_colours) == 1:
        master.at[index,'primary_colour'] = clean_colours[0]
    elif len(clean_colours) == 2:
        master.at[index,'primary_colour'] = clean_colours[0]
        master.at[index,'secondary_colour'] = clean_colours[1]
    elif len(clean_colours) == 3:
        master.at[index,'primary_colour'] = clean_colours[0]
        master.at[index,'secondary_colour'] = clean_colours[1]
        master.at[index,'tertiary_colour'] = clean_colours[2]
    master.at[index,'colour_family'] = col_fam[col_fam['unique colour'] == clean_colours[0].strip()].values[0][1]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
master['first_grn_date'] = pd.to_datetime(master['first_grn_date'], format='%d-%b-%Y')

In [ ]:
master.to_csv('clean_db.csv',index=False)

In [ ]:
sizes = []
styles = []
unique_products = set()

for index, row in master.iterrows():
    if row['product_id'] not in unique_products:
        unique_products.add(row['product_id'])
        styles.append({
            'product_id': row['product_id'],
            'style_id': row['style_id'],
            'brand': row['brand'],
            'gender': row['gender'],
            'division': row['division'],
            'department': row['department'],
            'usage': row['usage'],
            'product': row['product'],
            'sub_product': row['sub_product'],
            'target_audience': row['target_audience'],
            'fit': row['fit'],
            'mrp': row['mrp'],
            'cost': row['cost'],
            'colour_family': row['colour_family'],
            'primary_colour': row['primary_colour'],
            'secondary_colour': row['secondary_colour'],
            'tertiary_colour': row['tertiary_colour'],
            'tags': row['tags'],
            'season': row['season'],
            'exclusive': row['exclusive'],
            'garment_pattern': row['garment_pattern'],
            'print_pattern_type': row['print_pattern_type'],
            'number_of_components': row['number_of_components'],
            'number_of_pockets': row['number_of_pockets'],
            'pocket_type': row['pocket_type'],
            'length': row['length'],
            'neck': row['neck'],
            'collar': row['collar'],
            'placket': row['placket'],
            'sleeve_length': row['sleeve_length'],
            'sleeve_type': row['sleeve_type'],
            'hemline': row['hemline'],
            'waist_rise': row['waist_rise'],
            'closure': row['closure'],
            'footwear_insole': row['footwear_insole'],
            'fabric_code': row['fabric_code'],
            'fabric_rate': row['fabric_rate'],
            'fabric_story': row['fabric_story'],
            'fabric_composition': row['fabric_composition'],
            'fabric_hsn': row['fabric_hsn'],
            'fabric_weave_pattern': row['fabric_weave_pattern'],
            'fabric_vendor': row['fabric_vendor'],
            'denim_cast': row['denim_cast'],
            'denim_wash': row['denim_wash'],
            'wash_care': row['wash_care'],
            'footwear_upper_material': row['footwear_upper_material'],
            'footwear_sole_material': row['footwear_sole_material'],
            'first_grn_date': row['first_grn_date'],
            'first_live_date': row['first_live_date'],
            'first_sold_date': row['first_sold_date'],
        })
    sizes.append({
        'ean':row['ean'],
        'myntra_id':row['myntra_id'],
        'ajio_id':row['ajio_id'],
        'ajio_ean':row['ajio_ean'],
        'product_id':row['product_id'],
        'standard_size':row['standard_size'],
        'barcode_size':row['barcode_size'],
        'garment_waist':row['garment_waist'],
        'inseam_length':row['inseam_length'],
        'to_fit_waist':row['to_fit_waist'],
        'across_shoulder':row['across_shoulder'],
        'chest':row['chest'],
        'front_length':row['front_length'],
        'to_fit_foot_length':row['to_fit_foot_length'],
        'shoe_weight':row['shoe_weight'],
    })

In [ ]:
styles = pd.DataFrame.from_records(styles)
sizes = pd.DataFrame.from_records(sizes)

In [ ]:
styles.number_of_pockets.replace('Multiple Pockets', -1, inplace=True)
styles['status'] = 'Approved'
styles['created_at'] = '2023-01-11'
styles['created_by'] = 1
styles['deleted'] = False

In [ ]:
styles.to_csv('styles.csv', index=False)
sizes.to_csv('sizes.csv', index=False)